In [ ]:
import cv2
import mediapipe as mp
import torch
import numpy as np
from ultralytics import YOLO

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh_detector = mp_face_mesh.FaceMesh()

# Load YOLOv8 model
model_path = "yolov8n.pt"  # Ensure you have the trained model
face_detection_model = YOLO(model_path)

# Open webcam
webcam = cv2.VideoCapture(0)

while True:
    is_frame_captured, frame = webcam.read()
    if not is_frame_captured:
        break
    
    # YOLOv8 face detection
    detection_results = face_detection_model(frame)
    detected_faces = detection_results[0].boxes.xyxy.cpu().numpy()
    
    for face in detected_faces:
        x_min, y_min, x_max, y_max = map(int, face)
        face_region = frame[y_min:y_max, x_min:x_max]
        
        # Convert face region to RGB for MediaPipe processing
        face_region_rgb = cv2.cvtColor(face_region, cv2.COLOR_BGR2RGB)
        face_mesh_results = face_mesh_detector.process(face_region_rgb)
        
        if face_mesh_results.multi_face_landmarks:
            for face_landmarks in face_mesh_results.multi_face_landmarks:
                for landmark in face_landmarks.landmark:
                    face_height, face_width, _ = face_region.shape
                    landmark_x = int(landmark.x * face_width) + x_min
                    landmark_y = int(landmark.y * face_height) + y_min
                    cv2.circle(frame, (landmark_x, landmark_y), 1, (0, 255, 0), -1)
        
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)
    
    # Display output
    cv2.imshow("Face Profiling", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcam.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'mediapipe'